# LLM Prompting Analysis for Emotion Intensity, Empathy, and Polarity

This notebook performs LLM-based analysis using three different prompting strategies:
- Zero-Shot Prompting
- Few-Shot Prompting
- Chain-of-Thought Prompting

In [1]:
import sys
import os
import pandas as pd
import json
from datetime import datetime

sys.path.append(os.path.join(os.getcwd(), '..', 'Scripts'))
from llm_prompting import zero_shot_prompt, few_shot_prompt, chain_of_thought_prompt, analyze_with_llm

## Load Conversations from Dataset

In [2]:
def load_conversations_from_dataset(dataset_path, num_conversations=5, min_utterances=10):
    df = pd.read_csv(dataset_path)
    
    conversations_data = []
    conversation_ids = df['conversation_id'].unique()
    
    for conv_id in conversation_ids[:num_conversations]:
        conv_df = df[df['conversation_id'] == conv_id].sort_values('turn_id')
        
        if len(conv_df) >= min_utterances:
            utterances = []
            labels = []
            
            for _, row in conv_df.iterrows():
                utterances.append(f"{row['speaker']}: {row['text']}")
                labels.append({
                    'emotion_intensity': row['Emotion'],
                    'empathy': row['Empathy'],
                    'polarity': row['EmotionalPolarity']
                })
            
            article_id = conv_df['article_id'].iloc[0]
            
            conversations_data.append({
                'id': int(conv_id),
                'article_id': int(article_id),
                'utterances': utterances,
                'ground_truth_labels': labels,
                'num_turns': len(utterances)
            })
    
    return conversations_data

DATASET_PATH = '../Dataset/trac2_CONVT_train.csv'
CONVERSATIONS = load_conversations_from_dataset(DATASET_PATH, num_conversations=5, min_utterances=10)

print(f"Loaded {len(CONVERSATIONS)} conversations")
for conv in CONVERSATIONS:
    print(f"  Conversation {conv['id']}: {conv['num_turns']} turns")

Loaded 5 conversations
  Conversation 1: 22 turns
  Conversation 2: 21 turns
  Conversation 3: 25 turns
  Conversation 4: 31 turns
  Conversation 5: 20 turns


## Run Analysis on All Conversations

In [3]:
def run_analysis():
    results = {
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "model": "llama-3.3-70b-versatile",
        "conversations": []
    }
    
    prompting_strategies = [
        ("Zero-Shot", zero_shot_prompt),
        ("Few-Shot", few_shot_prompt),
        ("Chain-of-Thought", chain_of_thought_prompt)
    ]
    
    output_lines = []
    output_lines.append("="*80)
    output_lines.append("LLM PROMPTING ANALYSIS FOR EMOTION INTENSITY, EMPATHY, AND POLARITY")
    output_lines.append(f"Model: llama-3.3-70b-versatile via Groq API")
    output_lines.append(f"Timestamp: {results['timestamp']}")
    output_lines.append(f"Dataset: WASSA 2024 Track 2 (trac2_CONVT_train.csv)")
    output_lines.append("="*80)
    output_lines.append("")
    
    for conv in CONVERSATIONS:
        print(f"\nProcessing Conversation {conv['id']}...")
        conv_result = {
            "id": conv['id'],
            "article_id": conv['article_id'],
            "num_turns": conv['num_turns'],
            "utterances": conv['utterances'],
            "ground_truth_labels": conv['ground_truth_labels'],
            "analyses": {}
        }
        
        output_lines.append(f"\n{'='*80}")
        output_lines.append(f"CONVERSATION {conv['id']}")
        output_lines.append(f"Article ID: {conv['article_id']} | Turns: {conv['num_turns']}")
        output_lines.append(f"{'='*80}")
        output_lines.append("\nUtterances:")
        for i, utt in enumerate(conv['utterances'], 1):
            output_lines.append(f"  {i}. {utt}")
        
        output_lines.append("\nGround Truth Labels (from dataset):")
        for i, label in enumerate(conv['ground_truth_labels'], 1):
            output_lines.append(f"  Turn {i}: Emotion={label['emotion_intensity']}, Empathy={label['empathy']}, Polarity={label['polarity']}")
        output_lines.append("")
        
        for strategy_name, strategy_func in prompting_strategies:
            print(f"  - Running {strategy_name} prompting...")
            prompt = strategy_func(conv)
            response = analyze_with_llm(prompt)
            
            conv_result["analyses"][strategy_name] = {
                "prompt": prompt,
                "response": response
            }
            
            output_lines.append(f"\n{'-'*80}")
            output_lines.append(f"{strategy_name} Prompting")
            output_lines.append(f"{'-'*80}")
            output_lines.append("\nPrompt:")
            output_lines.append(prompt)
            output_lines.append("\nResponse:")
            output_lines.append(response)
            output_lines.append("")
        
        results["conversations"].append(conv_result)
    
    output_file = "../LLM_output.txt"
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("\n".join(output_lines))
    
    print(f"\nAnalysis complete! Results saved to {output_file}")
    
    json_file = "../LLM_output.json"
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
    
    print(f"Structured results saved to {json_file}")
    
    return results

## Execute Analysis

In [4]:
print("Starting LLM prompting analysis...")
print("Make sure GROQ_API_KEY environment variable is set!\n")

results = run_analysis()

print("\n" + "="*80)
print("SUMMARY")
print("="*80)
print(f"Total conversations analyzed: {len(CONVERSATIONS)}")
print(f"Prompting strategies used: Zero-Shot, Few-Shot, Chain-of-Thought")
print(f"Total API calls made: {len(CONVERSATIONS) * 3}")

Starting LLM prompting analysis...
Make sure GROQ_API_KEY environment variable is set!


Processing Conversation 1...
  - Running Zero-Shot prompting...
  - Running Few-Shot prompting...
  - Running Chain-of-Thought prompting...

Processing Conversation 2...
  - Running Zero-Shot prompting...
  - Running Few-Shot prompting...
  - Running Chain-of-Thought prompting...

Processing Conversation 3...
  - Running Zero-Shot prompting...
  - Running Few-Shot prompting...
  - Running Chain-of-Thought prompting...

Processing Conversation 4...
  - Running Zero-Shot prompting...
  - Running Few-Shot prompting...
  - Running Chain-of-Thought prompting...

Processing Conversation 5...
  - Running Zero-Shot prompting...
  - Running Few-Shot prompting...
  - Running Chain-of-Thought prompting...

Analysis complete! Results saved to ../LLM_output.txt
Structured results saved to ../LLM_output.json

SUMMARY
Total conversations analyzed: 5
Prompting strategies used: Zero-Shot, Few-Shot, Chain-of-Thought

## View Sample Results

In [5]:
print("First conversation analysis sample:")
print(f"\nConversation ID: {results['conversations'][0]['id']}")
print(f"Article ID: {results['conversations'][0]['article_id']}")
print(f"Number of turns: {results['conversations'][0]['num_turns']}")
print(f"\nZero-Shot Response:")
print(results['conversations'][0]['analyses']['Zero-Shot']['response'])

First conversation analysis sample:

Conversation ID: 1
Article ID: 35
Number of turns: 22

Zero-Shot Response:
```json
{
  "emotion_intensity": 4,
  "empathy": 5,
  "polarity": 2,
  "reasoning": "The conversation revolves around a sad article about people struggling with natural disasters and poverty. Both persons express sadness, frustration, and disappointment, indicating a high emotion intensity score of 4. The empathy score is 5 because both persons are able to put themselves in the shoes of those struggling, expressing feelings of sadness and frustration on their behalf. They also show understanding and compassion towards the people affected. The polarity is negative (2) because the overall tone of the conversation is somber and critical, discussing the struggles and injustices faced by certain communities. However, there are also moments of hope and optimism, particularly when discussing the potential for positive change with the new generation of leaders."
}
```
